## GTFS
In order to compute accessibility based on GTFS data (General Transit Feed Specification, can by downloaded from ftp://199.203.58.18/), we will first perform basic processing of the data.
We are using the pandas library.

The code is based on the following structure of GTFS tables:
![GTFS Tables](../../input_data/GTFS_tables.PNG)

In [1]:
import pandas as pd
import datetime as dt
import pickle
from tqdm.auto import tqdm
tqdm.pandas()

DATA_PATH = '../../input_data/GTFS-28-Oct-19/'

In [2]:
OUTPUT_PATH = '../../output_data/'

In [3]:
DAY = dt.datetime(2019, 11, 3)

## Process Calendar - Get trips for a single day

In [5]:
# Load calendar
calendar_df = pd.read_csv(DATA_PATH + 'calendar.txt')
# Convert dates to python's datetime type
calendar_df['start_date'] = calendar_df['start_date'].apply(lambda x: dt.datetime.strptime(str(x), '%Y%m%d'))
calendar_df['end_date'] = calendar_df['end_date'].apply(lambda x: dt.datetime.strptime(str(x), '%Y%m%d'))

calendar_df.columns = ['service_id',
 'sunday',
 'monday',
 'tuesday',
 'wednesday',
 'thursday',
 'friday',
 'saturday',
 'start_date',
 'end_date']

calendar_df[:3]

,service_id,sunday,monday,tuesday,wednesday,thursday,friday,saturday,start_date,end_date
0,1,1,1,1,1,1,0,0,2019-10-27,2019-10-31
1,2,1,1,1,1,1,0,0,2019-11-01,2019-12-26
2,3,0,0,0,0,0,1,0,2019-11-01,2019-12-26


In [6]:
# Let's say we want all trips that occurred on the first Sunday after the feed was published 

# Filter so we only keep services that are active on Sunday.
sunday_services_df = calendar_df[calendar_df['sunday'] == 1][['service_id', 'start_date', 'end_date']]

# Keep only services that start during/before selected date
sunday_services_df = sunday_services_df[sunday_services_df['start_date'] <= DAY]

# Keep only services that end during/after selected date
sunday_services_df = sunday_services_df[sunday_services_df['end_date'] >= DAY]

## Process Trips

In [7]:
# Load trips
trips_df = pd.read_csv(DATA_PATH + 'trips.txt')

# Fix column names (some columns have special 'hudden' characters that we want to remove)
trips_df.columns = ['route_id', 'service_id', 'trip_id','trip_headsign', 'direction_is', 'shape_id']

trips_calendar_df = sunday_services_df.merge(trips_df, on='service_id', suffixes=('_calendar', '_trips'))
sunday_trips_df = trips_calendar_df.drop(['start_date', 'end_date'], axis=1)
sunday_trips_df[:3]

,service_id,route_id,trip_id,trip_headsign,direction_is,shape_id
0,2,1,30900053_011119,רכבת יבנה מזרח,0,97105.0
1,2,1,30900054_011119,רכבת יבנה מזרח,0,97105.0
2,2,1,30900055_011119,רכבת יבנה מזרח,0,97105.0


In [8]:
sunday_trips_df.nunique()

service_id        6742
route_id          6742
trip_id          99653
trip_headsign     1710
direction_is         2
shape_id          6665
dtype: int64

### We now have all trips that occurred on the selected date

## Process Stop Times
Note: This is pretty heavy compared to the rest of the tables

In [9]:
# Load stop times
stop_times_df = pd.read_csv(DATA_PATH + 'stop_times.txt')

# Get all trips departures by getting the minimal departure time for each trip
trips_start_times_df = stop_times_df.groupby('trip_id').agg({'departure_time': 'min'})

# Let's join the last two tabled to get the departure times of all sunday trips
sunday_departures_df = sunday_trips_df.merge(trips_start_times_df, on='trip_id', suffixes=('_departures', '_trips'))

## Process Stops

In [10]:
def convert_gtfs_time_to_datetime(gtfs_time):
    date = dt.datetime(2019, 11, 3)
    h, m, s = [int(x) for x in gtfs_time.split(':')]
    if h < 24:
        # This is a 'normal' situation, we can simply create a datetime object using the date we defined before
        return date + dt.timedelta(hours=h, minutes=m, seconds=s)
    # Otherwise we have a 'strange' time: it's after midnight
    new_date = date + dt.timedelta(days=1)
    return new_date + dt.timedelta(hours=h-24, minutes=m, seconds=s)

In [11]:
# Load stops
stops_df = pd.read_csv(DATA_PATH + 'stops.txt')

# Add stop code and zone id to stop times
stop_times_with_stop_codes_df = stop_times_df.merge(
    stops_df[['stop_id', 'stop_code', 'zone_id']], on='stop_id')

# Add stop code to stop times - Yulia's example
# stop_times_with_stop_codes_df = stop_times_df.merge(
#     stops_df[['stop_id', 'stop_code']], on='stop_id')


stop_times_with_stop_codes_df['departure_time'] = stop_times_with_stop_codes_df[
    'departure_time'].apply(convert_gtfs_time_to_datetime)

## Construct Nodes

In [12]:
# We want to (right) join this table with stop_times in order to get the sunday stop times with trip departure time.
sunday_nodes_df = stop_times_df.merge(sunday_departures_df, how='right', on='trip_id', suffixes=('_stop', '_trip_departure'))

# Remove some columns to clear the data
sunday_nodes_df = sunday_nodes_df.drop(['pickup_type', 'shape_dist_traveled', 
                                        'drop_off_type', 'service_id', 'direction_is', 'shape_id'], 
                                       axis=1)

# Add stops data to nodes
nodes_df = sunday_nodes_df.merge(stops_df, on='stop_id', suffixes=('_node', '_stop'))
nodes_df = nodes_df.drop(['stop_desc', 'stop_name', 'zone_id', 'parent_station', 'location_type'],axis=1)

# Fot Yulia's example
# Remove some columns to clear the data
# sunday_nodes_df = sunday_nodes_df.drop(['pickup_type',
#                                         'drop_off_type', 'service_id', 'direction_is', 'shape_id'], 
#                                        axis=1)
# nodes_df = sunday_nodes_df.merge(stops_df, on='stop_id', suffixes=('_node', '_stop'))
# nodes_df = nodes_df.drop(['stop_desc', 'stop_name', 'parent_station', 'location_type'],axis=1)

nodes_df[:3]

,trip_id,arrival_time,departure_time_stop,stop_id,stop_sequence,route_id,trip_headsign,departure_time_trip_departure,stop_code,stop_lat,stop_lon
0,10021427_271019,00:00:00,00:00:00,36133,1,10376,כרמיאל_גבעת רם,00:00:00,47915,32.791771,35.034289
1,10021428_271019,01:00:00,01:00:00,36133,1,10376,כרמיאל_גבעת רם,01:00:00,47915,32.791771,35.034289
2,10021429_271019,02:00:00,02:00:00,36133,1,10376,כרמיאל_גבעת רם,02:00:00,47915,32.791771,35.034289


In [13]:
# Convert GTFS times to match "real-world time".
nodes_df['arrival'] = nodes_df['arrival_time'].apply(convert_gtfs_time_to_datetime)
nodes_df['departure'] = nodes_df['departure_time_stop'].apply(convert_gtfs_time_to_datetime)

In [14]:
nodes_df.head(3)

,trip_id,arrival_time,departure_time_stop,stop_id,stop_sequence,route_id,trip_headsign,departure_time_trip_departure,stop_code,stop_lat,stop_lon,arrival,departure
0,10021427_271019,00:00:00,00:00:00,36133,1,10376,כרמיאל_גבעת רם,00:00:00,47915,32.791771,35.034289,2019-11-03 00:00:00,2019-11-03 00:00:00
1,10021428_271019,01:00:00,01:00:00,36133,1,10376,כרמיאל_גבעת רם,01:00:00,47915,32.791771,35.034289,2019-11-03 01:00:00,2019-11-03 01:00:00
2,10021429_271019,02:00:00,02:00:00,36133,1,10376,כרמיאל_גבעת רם,02:00:00,47915,32.791771,35.034289,2019-11-03 02:00:00,2019-11-03 02:00:00


## Some Stats on the Overall Nodes For the Day 

In [15]:
nodes_df.nunique()

trip_id                          99653
arrival_time                     88667
departure_time_stop              88667
stop_id                          27622
stop_sequence                      117
route_id                          6742
trip_headsign                     1710
departure_time_trip_departure     1202
stop_code                        27273
stop_lat                         26961
stop_lon                         26795
arrival                          88667
departure                        88667
dtype: int64

In [20]:
# TODO: add node_id according to index, and save in pkl

In [16]:
nodes_df.to_pickle(OUTPUT_PATH + 'all_nodes.pkl')

In [17]:
start_time = DAY + dt.timedelta(hours=8)
end_time = start_time + dt.timedelta(hours=1, minutes=30)

morning_nodes_df = nodes_df[nodes_df['arrival'] > start_time][nodes_df['arrival'] < end_time]
morning_nodes_df.head(3)

d:\users\bensdani\venvs\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


,trip_id,arrival_time,departure_time_stop,stop_id,stop_sequence,route_id,trip_headsign,departure_time_trip_departure,stop_code,stop_lat,stop_lon,arrival,departure
18,14824097_271019,08:02:40,08:02:40,2356,2,1606,נוף הגליל_שלום,08:00:00,41476,32.793214,35.038925,2019-11-03 08:02:40,2019-11-03 08:02:40
19,14824098_271019,09:02:40,09:02:40,2356,2,1606,נוף הגליל_שלום,09:00:00,41476,32.793214,35.038925,2019-11-03 09:02:40,2019-11-03 09:02:40
36,19590744_271019,08:14:17,08:14:17,2356,2,16379,טבריה_תחנה מרכזית,08:10:00,41476,32.793214,35.038925,2019-11-03 08:14:17,2019-11-03 08:14:17


In [18]:
morning_nodes_df.nunique()

trip_id                          13765
arrival_time                      5399
departure_time_stop               5399
stop_id                          25713
stop_sequence                      117
route_id                          3793
trip_headsign                     1127
departure_time_trip_departure      199
stop_code                        25383
stop_lat                         25107
stop_lon                         24961
arrival                           5399
departure                         5399
dtype: int64

In [24]:
morning_nodes_df = morning_nodes_df.reset_index()

In [25]:
morning_nodes_df['node_id'] = morning_nodes_df.index

In [26]:
morning_nodes_df.head(3)

,index,trip_id,arrival_time,departure_time_stop,stop_id,stop_sequence,route_id,trip_headsign,departure_time_trip_departure,stop_code,stop_lat,stop_lon,arrival,departure,node_id
0,18,14824097_271019,08:02:40,08:02:40,2356,2,1606,נוף הגליל_שלום,08:00:00,41476,32.793214,35.038925,2019-11-03 08:02:40,2019-11-03 08:02:40,0
1,19,14824098_271019,09:02:40,09:02:40,2356,2,1606,נוף הגליל_שלום,09:00:00,41476,32.793214,35.038925,2019-11-03 09:02:40,2019-11-03 09:02:40,1
2,36,19590744_271019,08:14:17,08:14:17,2356,2,16379,טבריה_תחנה מרכזית,08:10:00,41476,32.793214,35.038925,2019-11-03 08:14:17,2019-11-03 08:14:17,2


In [27]:
morning_nodes_df.to_pickle(OUTPUT_PATH + 'morning_nodes.pkl')

## Filter only Tel Aviv Metropolitan stops

Let's get all Tel Aviv (TLV) stops 

In [ ]:
for zone in stops_df['zone_id'].unique():
    print(zone)

In [ ]:
test_stops = stops_df[stops_df['zone_id'] == 6900]
test_stops.to_csv(OUTPUT_PATH + 'test_stops.csv')

In [ ]:
test_stops.shape

In [ ]:
stops_df.shape

In [ ]:
# Let's see how manu NaN zones we have:
stops_df['zone_id'].isna().sum()

In [ ]:
# I think zone 210 is tel aviv metropolitan area (only the small surrounding part, we would need to extend for our real computations). 
# We need to filter only trips that contain stops (and then only nodes with those stops) that are in this zone.
tlv_stops_df = stops_df[stops_df['zone_id'] == 210]

In [ ]:
tlv_stops_df.shape

Now we need to get the TLV stop times, in order to find all trips that include TLV stops

In [ ]:
# stop_times_df
tlv_stop_times_df = stop_times_df.merge(tlv_stops_df[['stop_id']], on='stop_id', how='inner')

In [ ]:
tlv_stop_times_df.shape

In [ ]:
stop_times_df.shape

Next, the TLV stop times include all actual trip ids with TLV stops in them. Let's find relevant trip in the Sunday's trips we're examining

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
# We want to get unique trips which pass through TLV, then we will use these trips to filter 
# only nodes that pass through TLV.
#
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

tlv_trips_df = tlv_stop_times_df[['trip_id']].drop_duplicates(subset ="trip_id", 
                     keep = False, inplace = False)

In [ ]:
tlv_trips_df.head()

In [ ]:
tlv_trips_df.shape

Finally, let's save only nodes that include TLV trips

In [ ]:
tlv_nodes_df = nodes_df.merge(tlv_trips_df[['trip_id']], on='trip_id', how='inner')

In [ ]:
nodes_df.shape

In [ ]:
tlv_nodes_df.shape

In [ ]:
tlv_nodes_df.to_csv('tlv_nodes.csv')